# Structure from Motion

### The Projective space

Points in $ \mathbb{P}^2 $ are represented as

$$ \mathbf{x} = \begin{bmatrix}
x & y & w
\end{bmatrix}^T
$$

The equivalent point in $ \mathbb{R}^2 $ is

$$ \mathbf{x} = \begin{bmatrix}
x / w & y / w
\end{bmatrix}^T
$$

For points with $w \neq 0$. Points with $w=0$ is called points as infinity.

Lines are represented as

$$
\mathbf{x}^T\mathbf{l} = 0
$$

Where 

$$ \mathbf{l} = \begin{bmatrix}
a & b & c
\end{bmatrix}^T
$$


### Null space

Finding any non-zero vector $\mathbf{x}$ satifying the equation

$$
A\mathbf{x} = \mathbf{0}
$$

The nullspace is only determined up to a scale.

In [1]:
import numpy as np
from numpy.linalg import svd

def nullspace(A, atol=1e-13, rtol=0):
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

A = np.array([[1, 0],
              [0, 1]])
print(nullspace(A))

A = np.array([[1, 0],
              [0, 0]])
n = nullspace(A)
print(n)
print(A @ n)
k = 17
print(A @ (k * n))


[]
[[0.]
 [1.]]
[[0.]
 [0.]]
[[0.]
 [0.]]


### Projective camera

A projective camera $\texttt{P}$ transforms a world point $\mathbf{X}$ into a image point $\mathbf{x}.$

$$\mathbf{x} = \texttt{P}\mathbf{X}$$

Where $\texttt{P}$ is a 3x4 matrix and the world point $\mathbf{X}$ is a 4-vector while the imgage point $\mathbf{x}$ is a 3-vector.

In [2]:
P = np.array([
    [20,  0, 0, -10],
    [ 0, 20, 0, -10],
    [ 0,  0, 1, 0],
])
X = [3, 4, 10, 1]
x = P @ X
print(x / x[2])

[5. 7. 1.]


The camera matrix $\texttt{P}$ can be decomposed into the calibration matrix $\texttt{K}$ and a rotation and translation pair like so

$$
\texttt{P} = \texttt{K}
\begin{bmatrix}
\texttt{R} & \mathbf{t}
\end{bmatrix}
$$

Sometimes the camera centre $\tilde{C}$ is needed expicitly

$$
\mathbf{t} = -\texttt{R}\tilde{C}
$$

In [3]:
K = np.array([
    [10, 0, 50],
    [0, 10, 50],
    [0, 0, 1]
])
R = np.eye(3)
t = np.array([10, 10, 0])
P = K @ np.hstack([R, t[:, None]])
print(P)

[[ 10.   0.  50. 100.]
 [  0.  10.  50. 100.]
 [  0.   0.   1.   0.]]


### Calibration matrix

In [4]:
from ipywidgets import interact
from ipycplot import Plot3D, rectangle
import numgl
from numpy.linalg import inv

width, height = 320, 200

n = 10
x, y = np.meshgrid(np.linspace(0, width, n), np.linspace(0, height, n))
w = np.ones(x.shape)
# points in the image plane
p1 = np.stack([x, y, w], axis=-1).reshape((-1, 3))

plot = Plot3D()
plot.scale = np.array([200, 200, 1])
target = np.array([width/2, height/2, 0])
plot.camera.lookat(target=target)
def update_focal(f, ay, px, py):
    K = np.array([
        [f, 0, px],
        [0, f, py],
        [0, 0, 1],
    ])
    p2 = p1 + numgl.normalized(inv(K) @ p1.T, axis=0).T * 50
    lines = np.hstack([p1, p2]).reshape(-1, 3)
    
    r = 600
    eye = np.array([r * np.sin(ay), 0, r * np.cos(ay)])
    plot.camera.lookat(eye=eye + target)
    
    plot.canvas.clear()
    plot.lines(lines)
    plot.lines(rectangle(width, height))

interact(update_focal, f=(50, 1000), ay=(-np.pi/2, np.pi/2), px=(0, width), py=(0, height))
plot.show()

interactive(children=(IntSlider(value=525, description='f', max=1000, min=50), FloatSlider(value=0.0, descript…

RoughCanvas()

### Epipolar Geometry
The geometry of two cameras depicting the same scene. 
* The _epipolar point_ is the image of the camera center of the other image. It may lie outside the visible image.
* The _epipolar line_ is the line between the two camera centers.


In [5]:
import numpy as np
import numgl
from ipywidgets import interact
from ipycplot import Plot3D, rectangle, transform


width, height = 320, 200

# world points
n = 10
X = np.random.normal(size=(n, 3), scale=100)

def rt(pose):
    R, C = pose[0:3, 0:3], pose[0:3, 3][:, None]
    t = -R.T @ C
    return R, t

plot = Plot3D()
plot.scale = np.array([80, 80, 1])

f = 350
K = np.array([
    [f, 0, width/2],
    [0, f, height/2],
    [0, 0, 1],
])

left_pose =  numgl.translate((-190,    0, 500)) @ numgl.roty(2.8)
right_pose = numgl.translate(( 330, -100, 500)) @ numgl.roty(3.7)
P1 = K @ np.hstack(rt(left_pose))
P2 = K @ np.hstack(rt(right_pose))

def mask_points(x, width, height):
    mask = (x[:, 0] >= 0) * (x[:, 0] < width) * (x[:, 1] >= 0) * (x[:, 1] < height)
    return x[mask]

def update(ay):
    center = np.array([0, 150, 0])
    target = center
    r = 1100
    eye = center + np.array([r * np.sin(ay), 0, r * np.cos(ay)]) + np.array([0, -300, 0])
    plot.camera.lookat(eye=eye, target=target)

    plot.canvas.clear()

    # plot world points
    plot.circles(X)    
    
    # project points in left camera
    x1 = transform(P1, X)  # transform world points using camera
    x1 = mask_points(x1, width, height)  # mask out points outside screen
    x1 = np.hstack([x1, np.zeros((x1.shape[0], 1))])  # add z=0 coordinate
    plot.circles(transform(left_pose, x1))  # plot projected points for camera
    plot.lines(transform(left_pose, rectangle(width, height)))

    # project points in right camera
    x2 = transform(P2, X)  # transform world points using camera
    x2 = mask_points(x2, width, height)  # mask out points outside screen
    x2 = np.hstack([x2, np.zeros((x2.shape[0], 1))])  # add z=0 coordinate
    plot.circles(transform(right_pose, x2))  # plot projected points for camera
    plot.lines(transform(right_pose, rectangle(width, height)))
    
interact(update, ay=(-np.pi/2, np.pi/2))

plot.show()

interactive(children=(FloatSlider(value=0.0, description='ay', max=1.5707963267948966, min=-1.5707963267948966…

RoughCanvas()

### The Fundamental Matrix
The fundamental matrix maps points in the left image to lines in the right image.

$$
\mathbf{l'} = \texttt{F}\mathbf{x}
$$

A point in the left image $\mathbf{x}$ corresponding to the same world point $\mathbf{X}$ will lie on the epipolar line $\mathbf{l'}$ in the right image, so we have

$$
\mathbf{x'}^T\texttt{F}\mathbf{x} = \mathbf{0}
$$

It can be directly computed from two cameras like so

$$
\texttt{F} =
\begin{bmatrix}
\mathbf{e}'
\end{bmatrix}_\times 
\texttt{P}'\texttt{P}^+
$$

In [6]:
from numpy.linalg import pinv

def skew(x):
    return np.array([
        [    0, -x[2],  x[1]],
        [ x[2],     0, -x[0]],
        [-x[1],  x[0],    0]])

def from_cameras(P1, P2):
    C1 = nullspace(P1).flatten()  # camera center
    e2 = P2 @ C1  # epipole
    return skew(e2) @ P2 @ pinv(P1)

F = from_cameras(P1, P2)
print(F)

[[-1.46447971e-01 -4.81326969e-01  1.12239257e+02]
 [-2.81864098e-01 -2.55593651e-14  3.38446758e+02]
 [ 1.09432008e+01 -1.82613145e+02 -3.72723058e+04]]


In [7]:
print(nullspace(F))
print(nullspace(F.T))

[[ 9.93997891e-01]
 [-1.09396108e-01]
 [ 8.27818002e-04]]
[[ 0.84917653]
 [-0.52810436]
 [-0.00223824]]


In [8]:
def rank(A, tol=1e-13):
    _, s, _ = svd(A)
    return np.sum(s > tol)

rank(F)

2

In [9]:
from numpy import cross
from ipycplot import Plot2D, homogenize, dehomogenize
def from_implicit(lines, width, height):
    """Create a line segment from the intersection of an implicit line with a rectangle"""
    result = np.empty((0, 2))
    for l in lines:
        left = np.array([1, 0, 0])  # x = 0
        right = np.array([1, 0, -(width - 1)])  # x = width
        top = np.array([0, 1, 0])  # y = 0
        bottom = np.array([0, 1, -(height - 1)])  # y = height
        intersections = np.array([
            cross(l, left), cross(l, right), cross(l, top), cross(l, bottom),
        ])
        masked = mask_points(dehomogenize(intersections.T).T, width, height)
        result = np.append(result, masked, axis=0)
    return result

n = 7
x1 = np.hstack([np.random.rand(n, 2) * np.array([width, height]), np.ones((n, 1))])

plot = Plot2D()
plot.lines(from_implicit((F @ x1.T).T, width, height))
plot.show()

RoughCanvas()

### Fundamental Matrix from correspondences
Using $\mathbf{x'}^T\texttt{F}\mathbf{x} = \mathbf{0}$ we can compute $\texttt{F}$ from image correspondences alone. For a single correspondence $\mathbf{x} \leftrightarrow \mathbf{x}'$ where $\mathbf{x} = \left [ x \,  y \, 1  \right ]^T$ and $\mathbf{x}' = \left [ x '\,  y' \, 1  \right ]^T$ we can expand the inner products like so

$$
x'xf_{11} + x'yf_{12} + xf_{13} + y'xf_{21} + y'yf_{22} + y'f_{23} + xf_{31} + yf_{32} + f_{33} = 0
$$

If we introduce a vector $ \mathbf{f} = \left [ f_{11}, f_{12}, f_{13}, f_{21}, f_{22}, f_{23}, f_{31}, f_{32}, f_{33} \right ]^T $ it can be written as 

$$
\left [ x'x, x'y, x', y'x, y'y, y', x, y, 1  \right ] \mathbf{f} = \mathbf{0}
$$

Given several image correspondences we can stack these like so
$$
A\mathbf{f} =
\begin{bmatrix}
x_1'x_1 & x_1'y_1 & x_1' & y_1'x_1 & y_1'y_1 & y_1' & x_1 & y_1 & 1 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
x_n'x_n & x_n'y_n & x_n' & y_n'x_n & y_n'y_n & y_n' & x_n & y_n & 1 \\
\end{bmatrix} \mathbf{f}  = \mathbf{0}
$$



In [10]:
from numpy.linalg import norm
#from ipycplot import transform

def normalizing_transform(p):
    """Isotropic point normalization. Returns a transform that transform to mean 0, 0 and mean norm=sqrt(2)"""
    d = p.shape[1]
    c = np.mean(p, axis=0)
    s = np.mean(norm(p - c[None:], axis=-1))
    T = np.diag(np.append(np.repeat(s / np.sqrt(2), d), 1))
    T[:d, d] = c
    return inv(T)

x1 = transform(P1, X)  # transform world points using camera
x2 = transform(P2, X)  # transform world points using camera

# compute F
def from_correspondences(x1, x2):
    # compute normalizing transforms
    T1 = normalizing_transform(x1)
    T2 = normalizing_transform(x2)
    nx1 = transform(T1, x1)
    nx2 = transform(T2, x2)
    A = np.vstack([(x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1) for (x1, y1), (x2, y2) in zip(nx1, nx2)])

    #f = nullspace(A)
    u, s, vh = svd(A)
    f = vh[-1].T
    
    F = f.reshape((3, 3))    
    
    # force rank 2
    u, s, vh = svd(F)
    s[-1] = 0
    F = u @ np.diag(s) @ vh
    # denormalize F matrix
    return T2.T @ F @ T1

F = from_correspondences(x1, x2)
print(F)


[[-6.27769814e-06 -2.06327572e-05  4.81129352e-03]
 [-1.20825008e-05  3.63602002e-20  1.45079961e-02]
 [ 4.69095684e-04 -7.82796915e-03 -1.59772978e+00]]


### Triangulation

From the projection of world points $\mathbf{X}$ we have
$$
\mathbf{x} = \texttt{P}\mathbf{X} \\
\mathbf{x}' = \texttt{P}'\mathbf{X}
$$

The scale factor can be eliminated by using cross product, e.g. for the left image $\mathbf{x} \times (\texttt{P}\mathbf{X}) = \mathbf{0}$

In [11]:
def triangulate(P1, P2, x1, x2):
    Xs = []
    for (x1, y1), (x2, y2) in zip(x1, x2):
        A = np.array([
            x1 * P1[2, :] - P1[0, :],
            y1 * P1[2, :] - P1[1, :],
            x2 * P2[2, :] - P2[0, :],
            y2 * P2[2, :] - P2[1, :],
        ])
        u, s, vh = svd(A)
        Xs.append(vh[-1])
    return np.array(Xs)
        
Xr = triangulate(P1, P2, x1, x2)  # triangulate points
print(dehomogenize(P1 @ Xr.T).T)  # project back into image
print(x1)  # original image points

[[135.76628475  80.35837532]
 [276.970937   145.26482271]
 [237.9954386   26.69987338]
 [ 76.48781544 121.64586313]
 [159.73281201 227.33188613]
 [ 13.81023537  97.81116943]
 [214.76775148  97.02865852]
 [136.29244823 119.76802765]
 [290.47038331  72.9365592 ]
 [232.41853227  24.13531362]]
[[135.76628475  80.35837532]
 [276.970937   145.26482271]
 [237.9954386   26.69987338]
 [ 76.48781544 121.64586313]
 [159.73281201 227.33188613]
 [ 13.81023537  97.81116943]
 [214.76775148  97.02865852]
 [136.29244823 119.76802765]
 [290.47038331  72.9365592 ]
 [232.41853227  24.13531362]]


### Extracting cameras

In [12]:
from numpy.linalg import det

def depth(P, X):
    """Computes the depth of a world point X given a camera P"""
    M = P[:3, :3]
    w = (P @ X).flatten()[2]
    W = X.flatten()[3]
    return np.sign(det(M)) * w / (W * norm(M[-1]))  # or M[:, 2] ?

def is_infront(P, X):
    return depth(P, X) > 0

def extract_cameras(K1, K2, F, x1_test, x2_test):
    W = np.array([
        [0, -1, 0],
        [1,  0, 0],
        [0,  0, 1],
    ])  # orthonogal (W.T@W = W@W.T = I)
    Z = np.array([
        [ 0, 1, 0],
        [-1, 0, 0],
        [ 0, 0, 0],
    ])  # skew-symetric (-Z = Z.T)
    E = K2.T @ F @ K1
    U, s, Vh = svd(E)
    u3 = U.T[-1]
    
    alternatives = [
        (U @ W @ Vh, u3),
        (U @ W @ Vh, -u3),
        (U @ W.T @ Vh, u3),
        (U @ W.T @ Vh, -u3),
    ]

    P1 = K1 @ np.hstack([np.eye(3), np.zeros((3, 1))])
    for R, t in alternatives:
        P2 = K2 @ np.hstack([R, t[:, None]])
        X_test = triangulate(P1, P2, x1_test[None, :], x2_test[None, :]).flatten()
        if is_infront(P1, X_test) and is_infront(P2, X_test):
            return P1, P2

P1, P2 = extract_cameras(K, K, F, x1[3], x2[3])
P2

array([[ 9.22311834e+01, -3.14481112e-13,  3.73622013e+02,
        -2.02047684e+02],
       [-7.83326910e+01,  3.50000000e+02,  6.21609968e+01,
         1.25653807e+02],
       [-7.83326910e-01, -9.26391209e-16,  6.21609968e-01,
         5.32552022e-01]])

### Computing the Calibration matrix
Given a set of corresponding image and world points $\mathbf{x}_i \leftrightarrow \mathbf{X}_i $ compute the projective camera $\texttt{P}$. We know the world point is projected as $\mathbf{x} = \texttt{P}\mathbf{X}$. Again using the identity $ \mathbf{v} \times \mathbf{v} = \mathbf{0} $ we can write.

$$
\begin{bmatrix}
\mathbf{0}^T & -w_i\mathbf{X}_i^T & y_i\mathbf{X}_i^T  \\ 
w_i\mathbf{X}_i^T & \mathbf{0}^T & -x_i\mathbf{X}_i^T  \\ 
y_i\mathbf{X}_i^T & x_i\mathbf{X}_i^T & \mathbf{0}^T
\end{bmatrix} \mathbf{p} = \mathbf{0}
$$

Where $\mathbf{x}_i = \left [ x_i, y_i, w_i \right ] ^T $ and $\mathbf{p}$ is the elements of $\texttt{P}$.

Furthermore there are only two lineary independent equation and one can be omitted, giving two equations for each correspondance. The $\texttt{P}$ matrix have 11 degrees of freedom (12 for the elements or $\texttt{P}$, minus one for overall scale) so 5½ correspondances are needed, where only the x (or y) coordinate of the last image point need to be know.


In [13]:
def compute_camera(x, X):
    T = normalizing_transform(x)
    U = normalizing_transform(X)
    w = 1
    rows = []
    for (x, y), X in zip(transform(T, x), transform(U, X)):
        Xh = np.append(X, 1)
        rows.append(np.hstack([np.zeros(4), -w * Xh, y * Xh]))
        rows.append(np.hstack([w * Xh, np.zeros(4), -x * Xh]))
    #A = np.array(rows[:-1])  # skip last element
    A = np.array(rows)
    u, s, vh = svd(A)
    P = vh[-1].reshape((3, 4))
    return inv(T) @ P @ U  # denormalize

x1 = transform(P1, X)
#P = compute_camera(x1[0:6], X[0:6])
P = compute_camera(x1, X)
print(transform(P, X))
print(x1)

[[ 2.18677686e+02  1.64056236e+02]
 [-5.15283657e+02  3.73109848e+02]
 [-8.66357434e+02 -8.29402940e+02]
 [ 4.83067511e+02  1.56018156e+02]
 [ 3.76218343e+02 -1.56470115e+03]
 [-2.25492963e+04  4.42133055e+02]
 [-2.13219306e+02  7.78513006e+01]
 [ 1.28859548e+02 -7.08690685e+01]
 [ 1.25745606e-01  4.86385551e+01]
 [-6.57599928e+01 -2.07849653e+02]]
[[ 2.18677686e+02  1.64056236e+02]
 [-5.15283657e+02  3.73109848e+02]
 [-8.66357434e+02 -8.29402940e+02]
 [ 4.83067511e+02  1.56018156e+02]
 [ 3.76218343e+02 -1.56470115e+03]
 [-2.25492963e+04  4.42133055e+02]
 [-2.13219306e+02  7.78513006e+01]
 [ 1.28859548e+02 -7.08690685e+01]
 [ 1.25745606e-01  4.86385551e+01]
 [-6.57599928e+01 -2.07849653e+02]]


Using thee RQ-factorization $A = RQ$ for any matrix $A$ where $R$ is a _right_ (upper) triangular matrix and $Q$ is an orthonogal matrix,  we can find the Calibration Matrix from a projective camera $P$.

$$
\texttt{P} = \begin{bmatrix} \texttt{M} | -\texttt{M}\tilde{C} \end{bmatrix} = \begin{bmatrix} \texttt{KR} | -\texttt{KR}\tilde{C} \end{bmatrix}
$$

In [14]:
from scipy.linalg import rq

def decompose_camera(P):
    M = P[:3, :3]
    K, R = rq(M)
    T = np.diag(np.sign(np.diag(K)))
    if det(T) < 0:
        T[1, 1] *= -1
    K = K @ T
    K = K / K[-1, -1]  # normalize with K[2, 2] == 1
    C = inv(-M) @ P[:,3]
    return K, T @ R, C

decompose_camera(P)

(array([[ 3.50000000e+02,  6.76790241e-13,  1.60000000e+02],
        [ 0.00000000e+00, -3.50000000e+02,  1.00000000e+02],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[-1.00000000e+00, -1.37456338e-15, -5.03599125e-16],
        [-1.37456338e-15,  1.00000000e+00,  3.15631928e-16],
        [ 5.03599125e-16,  3.15631928e-16, -1.00000000e+00]]),
 array([ 1.27885835e-13,  1.38247921e-13, -1.04792445e-13]))

# Real Images


In [39]:
import ipywidgets
from ipycanvas import Canvas
import PIL

class Clicker:
    def __init__(self, filename):
        im = PIL.Image.open(filename)
        self.canvas = Canvas()
        self.scale = self.canvas.width / im.width
        self.im = im.resize((int(im.width * self.scale), int(im.height * self.scale)))
        self.canvas.put_image_data(np.array(self.im), 0, 0)
        self.canvas.on_mouse_down(self)
        self._clicks = []

    def __call__(self, x, y):
        self.canvas.stroke_style = 'white'
        self.canvas.stroke_circle(x, y, 5)
        self._clicks.append((x, y))

    def plot(self, clicks):
        self.canvas.stroke_style = 'white'
        self.canvas.stroke_circles(clicker1.scale * x1[:, 0], clicker1.scale * x1[:, 1], 5 * np.ones(len(x1)))
        
    def clicks(self):
        return np.array(self._clicks) / self.scale  # scale clicks back to image resolution

clicker = Clicker('data/calibration-target.jpeg')
clicker.canvas

Canvas()

In [17]:
# measured image points
xc = np.load('calibration-points.npy')
#xc = clicker.clicks()
#xc[:, 1] = height - xc[:, 1]  # negate y-coordinates
#print(2816, 1880)  # original resolution

# measured world points
XC = np.array([
    [0, 0, 17],  # top left
    [290, 0, 17],  # top right
    [290, 287, 17],  # bottom right
    [0, 287, 17],  # bottom left
    [290/2, 287/2, 17],  # mid point
    [0, 0, 0],  # top left (lower)
    [290, 0, 0],  # top right (lower)
    [290, 287, 0],  # bottom right (lower)
])

P_calibration = compute_camera(xc, XC)
P_calibration
decompose_camera(P_calibration)

(array([[ 1.66510775e+03, -2.99768229e+01,  3.62447692e+02],
        [ 0.00000000e+00, -1.54767174e+03, -1.28572941e+02],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[-0.44117229, -0.89498438,  0.06610571],
        [-0.42298704,  0.27234152,  0.86424074],
        [-0.79148529,  0.35331721, -0.49871633]]),
 array([1141.13955478, -293.01708214, 1022.96564233]))

In [42]:
clicker1 = Clicker('data/1.jpeg')
x1 = np.load('1.npy')
clicker1.plot(x1)
clicker1.canvas

Canvas()

In [52]:
clicker2 = Clicker('data/2.jpeg')
x2 = np.load('2.npy')
clicker2.plot(x2)
clicker2.canvas

Canvas()

In [ ]:
#x1 = clicker1.clicks()
#np.save('1.npy', x1)
#print(x1)

#x2 = clicker2.clicks()
#np.save('2.npy', x2)
#print(x2)

In [20]:
F = from_correspondences(x1, x2)
F

array([[ 1.27998147e-08,  1.09764908e-07, -1.30345206e-04],
       [ 6.55048038e-08, -1.15224107e-08, -1.64132369e-03],
       [ 1.05392905e-04,  1.50475198e-03, -5.19222591e-02]])

In [51]:
plot = Plot2D()
plot.image(np.array(clicker1.im))
plot.canvas.stroke_style = 'white'
plot.lines(from_implicit((F @ homogenize(clicker1.scale * x1.T)).T, clicker1.im.width, clicker1.im.width))
plot.show()

RoughCanvas()